In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Preprocess your X-ray images
image_size = (224, 224)
batch_size = 32

train_data_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
validation_data_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load the DenseNet121 model
base_model = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size[0], image_size[1], 3)
)

# Add your own classification layers
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Define the checkpoint callback
checkpoint_path = 'checkpoints'
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Define the early stopping callback
early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    verbose=1,
    restore_best_weights=True
)

# Train the model
epochs = 100

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size,
    epochs=epochs,
    callbacks=[checkpoint_callback, early_stopping_callback]
)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
29084464/29084464 [==============================] - 0s 0us/step
Epoch 1/100
163/163 [==============================] - ETA: 0s - loss: 0.1620 - accuracy: 0.9402
Epoch 1: val_accuracy improved from -inf to 0.82895, saving model to checkpoints
163/163 [==============================] - 242s 958ms/step - loss: 0.1620 - accuracy: 0.9402 - val_loss: 0.5258 - val_accuracy: 0.8289
Epoch 2/100
163/163 [==============================] - ETA: 0s - loss: 0.1039 - accuracy: 0.9636
Epoch 2: val_accuracy did not improve from 0.82895
163/163 [==============================] - 123s 753ms/step - loss: 0.1039 - accuracy: 0.9636 - val_loss: 1.0936 - val_accuracy: 0.7188
Epoch 3/100
163/163 [==============================] - ETA: 0s - loss: 0.1140 - accuracy: 0.9599
Epoch 3: val_accuracy improved from 0.82895 to 0.84375, saving model to checkpoints
163/163 [==============================] - 128s 783ms/step - loss: 0.1140 -